In [18]:
import sys
import os
import pandas as pd

os.chdir(os.path.abspath(".."))
sys.path.append(os.path.abspath(".."))

In [19]:
from dotenv import load_dotenv
import requests, json
load_dotenv()


email = "YOUR_EMAIL"      # J-Quants登録メール
password = "YOUR_PASS"    # パスワード
auth_payload = {"mailaddress": email, "password": password}
r = requests.post("https://api.jquants.com/v1/token/auth_user",
                  data=json.dumps(auth_payload))
refresh_token = os.getenv('JPX_API_KEY')

In [20]:
id_url = f"https://api.jquants.com/v1/token/auth_refresh?refreshtoken={refresh_token}"
r2 = requests.post(id_url)
id_token = r2.json().get("idToken")
print("ID Token:", id_token[:20], "...")  # トークンは長い文字列

ID Token: eyJraWQiOiJHQXNvU2xx ...


# J-Quants Standard API で日次株価を取得し Postgres に保存  
TOPIX500＋グロース市場を含む全銘柄（companies テーブルに登録済み）の  
2016‑01‑01 以降の日次株価を取得して **daily_quotes** テーブルへ UPSERT  

In [21]:
import os, json, time, logging
from datetime import datetime
from typing import Optional, List

import pandas as pd
import requests
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# ───────────── 環境変数読み込み ─────────────
load_dotenv()                         # .env を読み込む
JQ_USER   = os.getenv("JQUANTS_USER")
JQ_PASS   = os.getenv("JQUANTS_PASS")
JQ_TOKEN  = os.getenv("JPX_API_KEY")  # ← リフレッシュトークン
DB_USER   = os.getenv("POSTGRES_USER")
DB_PASS   = os.getenv("POSTGRES_PASSWORD")
DB_HOST   = os.getenv("POSTGRES_HOST", "localhost")
DB_PORT   = os.getenv("POSTGRES_PORT", "5432")
DB_NAME   = os.getenv("POSTGRES_DB")

# ───────────── ロギング ─────────────
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# ───────────── DB エンジン ─────────────
db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_url)

In [28]:
# Cell 3: 取得対象の証券コードを DB から取得（4〜5桁の数字のみ）
sql_codes = """
    SELECT DISTINCT security_code
      FROM companies
"""
codes_df = pd.read_sql(sql_codes, engine)
codes = codes_df["security_code"].astype(str).tolist()
print(f"対象コード数: {len(codes)}")

対象コード数: 1264


In [23]:
## 認証ヘルパ

# def get_id_token() -> str:
#     """環境変数にリフレッシュトークンがあればそれを使用、無ければメール/パスで取得"""
#     if JQ_TOKEN:                                  # リフレッシュトークン → ID トークン
#         resp = requests.post(
#             "https://api.jquants.com/v1/token/auth_refresh",
#             params={"refreshtoken": JQ_TOKEN}
#         )
#         resp.raise_for_status()
#         return resp.json()["idToken"]

#     # メール&パスワード認証 → リフレッシュトークン → ID トークン
#     auth_payload = {"mailaddress": JQ_USER, "password": JQ_PASS}
#     r = requests.post(
#         "https://api.jquants.com/v1/token/auth_user",
#         data=json.dumps(auth_payload),
#         headers={"Content-Type": "application/json"}
#     )
#     r.raise_for_status()
#     refresh = r.json()["refreshToken"]
#     r2 = requests.post(
#         "https://api.jquants.com/v1/token/auth_refresh",
#         params={"refreshtoken": refresh}
#     )
#     r2.raise_for_status()
#     return r2.json()["idToken"]

# ID_TOKEN = get_id_token()
# logger.info("✅ J‑Quants ID token acquired")
ID_TOKEN = id_token

In [29]:
# Cell 4: UPSERT 用 SQL と期間設定
DATE_FROM = "2016-01-01"
DATE_TO   = datetime.today().strftime("%Y-%m-%d")

insert_sql = text("""
INSERT INTO daily_quotes (
    code, date, open, high, low, close, volume, turnover_value,
    adjustment_factor, adjusted_open, adjusted_high, adjusted_low,
    adjusted_close, adjusted_volume, upper_limit, lower_limit
)
VALUES (
    :code, :date, :open, :high, :low, :close, :volume, :turnover_value,
    :adjustment_factor, :adjusted_open, :adjusted_high, :adjusted_low,
    :adjusted_close, :adjusted_volume, :upper_limit, :lower_limit
)
ON CONFLICT (code, date) DO UPDATE SET
    open              = EXCLUDED.open,
    high              = EXCLUDED.high,
    low               = EXCLUDED.low,
    close             = EXCLUDED.close,
    volume            = EXCLUDED.volume,
    turnover_value    = EXCLUDED.turnover_value,
    adjustment_factor = EXCLUDED.adjustment_factor,
    adjusted_open     = EXCLUDED.adjusted_open,
    adjusted_high     = EXCLUDED.adjusted_high,
    adjusted_low      = EXCLUDED.adjusted_low,
    adjusted_close    = EXCLUDED.adjusted_close,
    adjusted_volume   = EXCLUDED.adjusted_volume,
    upper_limit       = EXCLUDED.upper_limit,
    lower_limit       = EXCLUDED.lower_limit;
""")

In [30]:
# Cell 5: データ取得＆UPSERT のループ
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

success, fail = 0, 0
failed_codes = []

for code in tqdm(codes, desc="Upserting daily_quotes"):
    try:
        # J-Quants API からデータ取得（関数 fetch_daily_quotes は事前定義済み）
        df = fetch_daily_quotes(code, DATE_FROM, DATE_TO, ID_TOKEN)
        if df is None or df.empty:
            logger.warning(f"[{code}] データなし → スキップ")
            fail += 1
            failed_codes.append(code)
            continue

        # NaN → None に置換（BIGINT カラム挿入エラー回避）
        for col in ["volume", "turnover_value", "adjusted_volume"]:
            df[col] = df[col].where(df[col].notna(), None).astype("Int64")

        # レコード化 & UPSERT
        records = df.to_dict("records")
        with engine.begin() as conn:
            conn.execute(insert_sql, records)

        success += 1
        logger.info(f"[{code}] {len(records)} 件 upsert 成功")

    except DataError as de:
        logger.error(f"[{code}] DataError: {de}")
        fail += 1
        failed_codes.append(code)

    except Exception as e:
        logger.error(f"[{code}] その他エラー: {e}")
        fail += 1
        failed_codes.append(code)

logger.info(f"✅ 完了: 成功 {success} 銘柄, 失敗 {fail} 銘柄")
if failed_codes:
    logger.info(f"失敗銘柄一覧: {failed_codes}")

Upserting daily_quotes: 100%|█████████████████████████████████████████| 1264/1264 [40:27<00:00,  1.92s/it]
2025-07-03 01:03:14,380 [INFO] ✅ 完了: 成功 1264 銘柄, 失敗 0 銘柄
